In [1]:
library(dplyr)
library(tidyr)
library(stringr)
library(geohash)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [37]:
library(downloader)
# Blight Violations
dbvurl <- "https://d18ky98rnyall9.cloudfront.net/_97bd1c1e5df9537bb13398c9898deed7_detroit-blight-violations.csv?Expires=1484524800&Signature=hlHM02mABfvFUvErGgrV3u31Nr8GO6cwNPX1Z0wzrdY5o8q7faWEsmeoOcGE5p43iH3-WEtliSCduHMa~nrVcxeZpM-vvNhNXIU3D8m~0yBqrjbCpTeVdaC0IA-RgKNN1NfgMHoeSjSItNmjXcVfFq3U5X~xbNuv0DI4No8Qe80_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DBVFileName <- "detroit-blight-violations.csv"
if (!file.exists(DBVFileName)) 
    download(dbvurl, DBVFileName)
detBlightViol <- read.csv("detroit-blight-violations.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )   
    
# Detroit Demolition Permits 
ddpurl <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-demolition-permits.tsv?Expires=1487116800&Signature=A-ZtSkr8DZMpetuP4NNGPpz327wizHe9iw2SUTvlU5-eOJ1eTgnIflqgmfnaJJ-ultJMdawSbRD-tp5vtc45CnB1E9nMGwO-mFrca3z8bz3HKL09YgJEU2bM571hYeULyucNTcjejsQMmfZno8N2Qs9AwDigjlD1Ixy-pnyk2Ns_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DDPFileName <- "detroit-demolition-permits.tsv"
if (!file.exists(DDPFileName)) 
    download(ddpurl, DDPFileName)
detDemolitionPermit <- read.delim("detroit-demolition-permits.tsv", header=TRUE, allowEscapes=FALSE, sep="\t", na.strings="", comment.char="")

    
# Detroit 311 calls
d311url <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv?Expires=1487116800&Signature=i9Aq7nh9-0FvxpRKJLXXxN~XA~vVQ-Nk5AndCHEaoBY5vAf1CT8lt9xnVXwL5sbcMGD3LDIc~6LnB8daTqFgJFgKki86q885HsgiLH2-GnZy5zVmSEtkv0bPmMKwPbs10SNgNl7td~8CUb3MhmpUiQr1KjbeYTQYYZWUzJGZGNI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
D311FileName <- "detroit-311.csv"
if (!file.exists(D311FileName)) 
    download(d311url, D311FileName)
det311 <- read.csv("detroit-311.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )  

# Detroit Crime 
dcrurl <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv?Expires=1487116800&Signature=i9Aq7nh9-0FvxpRKJLXXxN~XA~vVQ-Nk5AndCHEaoBY5vAf1CT8lt9xnVXwL5sbcMGD3LDIc~6LnB8daTqFgJFgKki86q885HsgiLH2-GnZy5zVmSEtkv0bPmMKwPbs10SNgNl7td~8CUb3MhmpUiQr1KjbeYTQYYZWUzJGZGNI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DCRFileName <- "detroit-crime.csv"
if (!file.exists(DCRFileName)) 
    download(dcrurl, DCRFileName)
detCrime <- read.csv("detroit-crime.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )    
       

In [38]:
#dplyr mutate to extract (LAT,LNG) from ViolationAddress
detBlightViol <- detBlightViol %>%
    mutate(latlng = gsub(".*\\n", "", ViolationAddress)) %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    filter(latlng != 'character(0)') %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    separate(latlng, c('LAT', 'LNG'), ', ')
#head(detBlightViol)

#dplyr mutate to extract (LAT,LNG) from site_location
detDemolitionPermit <- detDemolitionPermit %>%
    mutate(latlng = gsub(".*\\n", "", site_location)) %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    filter(latlng != 'character(0)') %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    separate(latlng, c('LAT', 'LNG'), ', ')



Warning message:
"Too few values at 14 locations: 18, 23, 28, 88, 89, 94, 99, 144, 172, 189, 194, 206, 227, 335"

In [39]:
#dplyr filter to create gh_8 gh_7 and uid colums 
detBlightViol <- detBlightViol %>%
    mutate(gh_8 = gh_encode(as.numeric(LAT), as.numeric(LNG), 8),
           gh_7 = gh_encode(as.numeric(LAT), as.numeric(LNG), 7),
           uid = paste0('blightviol_',row_number()))

detDemolitionPermit <- detDemolitionPermit %>%
    mutate(gh_8 = gh_encode(as.numeric(LAT), as.numeric(LNG), 8),
           gh_7 = gh_encode(as.numeric(LAT), as.numeric(LNG), 7),
           uid = paste0('demolper',row_number()))

det311 <- det311 %>%
    mutate(gh_8 = gh_encode(as.numeric(lat), as.numeric(lng), 8),
           gh_7 = gh_encode(as.numeric(lat), as.numeric(lng), 7),
           uid = paste0('det311',row_number()))

detCrime <- detCrime %>%
    mutate(gh_8 = gh_encode(as.numeric(lat), as.numeric(lng), 8),
           gh_7 = gh_encode(as.numeric(lat), as.numeric(lng), 7),
           uid = paste0('detcrim',row_number()))

In [41]:
dbv_gh_8_grp <- detBlightViol %>%
    group_by(gh_8) %>%
    summarize(dbv_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(dbv_ngbr_8_ct))

ddp_gh_8_grp <- detDemolitionPermit %>%
    group_by(gh_8) %>%
    summarize(ddp_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(ddp_ngbr_8_ct))

d311_gh_8_grp <- det311 %>%
    group_by(gh_8) %>%
    summarize(d311_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(d311_ngbr_8_ct))

dcr_gh_8_grp <- detCrime %>%
    group_by(gh_8) %>%
    summarize(dcr_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(dcr_ngbr_8_ct))


#summary(dbv_gh_8_grp)
#summary(ddp_gh_8_grp)
#summary(d311_gh_8_grp)
#summary(dcr_gh_8_grp)

     gh_8           dbv_ngbr_8_ct      
 Length:67974       Min.   :    1.000  
 Class :character   1st Qu.:    1.000  
 Mode  :character   Median :    3.000  
                    Mean   :    4.528  
                    3rd Qu.:    5.000  
                    Max.   :21114.000  

     gh_8           ddp_ngbr_8_ct    
 Length:4596        Min.   :  1.000  
 Class :character   1st Qu.:  1.000  
 Mode  :character   Median :  1.000  
                    Mean   :  1.377  
                    3rd Qu.:  1.000  
                    Max.   :548.000  

     gh_8           d311_ngbr_8_ct  
 Length:15933       Min.   : 1.000  
 Class :character   1st Qu.: 1.000  
 Mode  :character   Median : 1.000  
                    Mean   : 1.235  
                    3rd Qu.: 1.000  
                    Max.   :90.000  

     gh_8           dcr_ngbr_8_ct   
 Length:15933       Min.   : 1.000  
 Class :character   1st Qu.: 1.000  
 Mode  :character   Median : 1.000  
                    Mean   : 1.235  
                    3rd Qu.: 1.000  
                    Max.   :90.000  

In [42]:
dbv_gh_7_grp <- detBlightViol %>%
    group_by(gh_7) %>%
    summarize(dbv_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(dbv_ngbr_7_ct))

ddp_gh_7_grp <- detDemolitionPermit %>%
    group_by(gh_7) %>%
    summarize(ddp_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(ddp_ngbr_7_ct))

d311_gh_7_grp <- det311 %>%
    group_by(gh_7) %>%
    summarize(d311_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(d311_ngbr_7_ct))

dcr_gh_7_grp <- detCrime %>%
    group_by(gh_7) %>%
    summarize(dcr_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(dcr_ngbr_7_ct))

summary(dbv_gh_7_grp)
summary(ddp_gh_7_grp)
summary(d311_gh_7_grp)
summary(dcr_gh_7_grp)

     gh_7           dbv_ngbr_7_ct     
 Length:15320       Min.   :    1.00  
 Class :character   1st Qu.:    6.00  
 Mode  :character   Median :   13.00  
                    Mean   :   20.09  
                    3rd Qu.:   25.00  
                    Max.   :21128.00  

     gh_7           ddp_ngbr_7_ct    
 Length:3417        Min.   :  1.000  
 Class :character   1st Qu.:  1.000  
 Mode  :character   Median :  1.000  
                    Mean   :  1.853  
                    3rd Qu.:  2.000  
                    Max.   :548.000  

     gh_7           d311_ngbr_7_ct  
 Length:9256        Min.   : 1.000  
 Class :character   1st Qu.: 1.000  
 Mode  :character   Median : 1.000  
                    Mean   : 2.126  
                    3rd Qu.: 3.000  
                    Max.   :95.000  

     gh_7           dcr_ngbr_7_ct   
 Length:9256        Min.   : 1.000  
 Class :character   1st Qu.: 1.000  
 Mode  :character   Median : 1.000  
                    Mean   : 2.126  
                    3rd Qu.: 3.000  
                    Max.   :95.000  

lat,lng,lat_error,lng_error
42.33161,-83.04789,8.583069e-05,0.0001716614
